In [20]:
import os 

In [22]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "jayraj2498"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "777e2be0b0c43fcc2efbc898716cbaebe35c912b"




In [3]:
%pwd

'e:\\Ds_p1\\DSP_End_KNN_prj\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\Ds_p1\\DSP_End_KNN_prj'

In [29]:
from dataclasses import dataclass 
from pathlib import Path 


@dataclass 
class ModelEvaluationConfig : 
    root_dir : Path 
    test_data_path : Path 
    model_path : Path 
    all_params : dict 
    metric_file_name : Path 
    target_column : str 
    mlflow_uri : str

In [30]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml , create_directories , save_json 

In [31]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet  # or self.params["ElasticNet"]
        schema = self.schema.TARGET_COLUMN  # or self.schema["TARGET_COLUMN"]["name"]

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
        root_dir=config.root_dir,
        test_data_path=config.test_data_path,
        model_path=config.model_path,
        all_params=params,
        metric_file_name=config.metric_file_name,
        target_column=schema.name , 
        mlflow_uri="https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow" )
        
        
        return model_evaluation_config   
        

In [32]:
import os 
import pandas as pd 
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score 
import mlflow
import numpy as np 
import joblib 
import mlflow.sklearn 
from urllib.parse import urlparse

In [ ]:
# # model evaluation component 

# class ModelEvaluation : 
#     def __init__(self, config:ModelEvaluationConfig) :
#         self.config= config 
        
        
#     def eval_metrics(self,actual,pred): 
#         rmse = np.sqrt(mean_squared_error(actual,pred)) 
#         mae = mean_absolute_error(actual,pred) 
#         r2 = r2_score(actual,pred) 
        
#         return rmse,mae,r2 
    
#     def log_into_mlflow(self) :
        
#         test_data = pd.read_csv(self.config.test_data_path)
#         model = joblib.load(self.config.model_path)  
        
#         test_x = test_data.drop([self.config.target_column], axis=1)
#         test_y = test_data[self.config.target_column] 
        
#         mlflow.set_registry_uri(self.config.mlflow_uri) 
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme 
        
        
#         with mlflow.start_run():
            
#             predicted_qualities = model.predict(test_x)

#             (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

#             # saving metrics locally
#             scores = {"rmse": rmse, "mae": mae, "r2": r2}
#             save_json(path=Path(self.config.metric_file_name), data=scores)

#             # log parameters
#             mlflow.log_params(self.config.all_params)

#             # log metrics (fixed: use log_metric, not log_metrics for single values)
#             mlflow.log_metric("rmse", rmse)
#             mlflow.log_metric("r2", r2)
#             mlflow.log_metric("mae", mae)

#             # model registry does not work with file store
#             if tracking_url_type_store != "file":
#                 mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
#             else:
#                 mlflow.sklearn.log_model(model, "model")

        
        

In [ ]:
# model evaluation component 
# it is the code to save model locally cuase mlflow model registry is not supported in dagshub

class ModelEvaluation : 
    def __init__(self, config:ModelEvaluationConfig) :
        self.config= config 
        
        
    def eval_metrics(self,actual,pred): 
        rmse = np.sqrt(mean_squared_error(actual,pred)) 
        mae = mean_absolute_error(actual,pred) 
        r2 = r2_score(actual,pred) 
        
        return rmse,mae,r2 
    
    def log_into_mlflow(self) :
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)  
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column] 
        
        mlflow.set_registry_uri(self.config.mlflow_uri) 
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme 
        


        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # log params
            mlflow.log_params(self.config.all_params)

            # log metrics
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            #  disable log_model (unsupported on DagsHub MLflow)
            # mlflow.sklearn.log_model(model, "model")

            #  instead, save model locally
            model_path = Path("artifacts/model_evaluation/saved_model.pkl")
            joblib.dump(model, model_path)
            print(f"Model saved locally at {model_path}")


In [34]:
try :
    config = ConfigurationManager() 
    model_eval_config = config.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(config=model_eval_config)
    model_evaluation.log_into_mlflow() 
    
except Exception as e :
    
    raise e 

[2025-09-28 20:29:43,805: INFO: common: yaml file : config\config.yaml loaded successfully]
[2025-09-28 20:29:43,812: INFO: common: yaml file : Params.yaml loaded successfully]
[2025-09-28 20:29:43,817: INFO: common: yaml file : schema.yaml loaded successfully]
[2025-09-28 20:29:43,819: INFO: common: created directory at : artifacts]
[2025-09-28 20:29:43,821: INFO: common: created directory at : artifacts/model_evaluation]
[2025-09-28 20:29:43,942: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /jayraj2498/DSP_End_KNN_prj.mlflow/api/2.0/mlflow/runs/create]
[2025-09-28 20:29:45,157: INFO: common: json file saved at : artifacts\model_evaluation\metrics.json]
Model saved locally at artifacts\model_evaluation\saved_model.pkl
🏃 View run smiling-bass-61 at: https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow/#/experiments/0/runs/574e5a58f20a47b696d1

In [48]:
import os 

os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow/api/mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "jayraj2498"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "777e2be0b0c43fcc2efbc898716cbaebe35c912b"


In [49]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow/api/mlflow


In [ ]:
"777e2be0b0c43fcc2efbc898716cbaebe35c912b" 
"https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow"
"https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow"